In [1]:
!pip install -q langchain_community
!pip install -q pypdf
!pip install -q -U langchain-text-splitters
!pip install -q -U langchain
!pip install -q -U bitsandbytes
!pip install -q -U accelerate
!pip install -q -U chromadb
!pip install -q -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.5/287.5 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.0/113.0 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 520.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from transformers import BitsAndBytesConfig
from langchain.prompts import PromptTemplate
from langchain_community.embeddings import LlamafileEmbeddings
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

model_name = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=nf4_config)

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [10]:
from langchain_community.document_loaders import PyPDFLoader

pdf = PyPDFLoader("/content/drive/MyDrive/llm/planet.pdf")
pages = pdf.load_and_split()


In [11]:
template = """
<s>[INST]<<SYS>>Answer the question only based on details in the context string below but dont mention about context.
If the answer is not in the context, reply "I dont know".
Context: {context}
<</SYS>>
Query: {query} [/INST]
"""

prompt = PromptTemplate.from_template(template)
fprompt = prompt.format(context="My give name is venkata ratnam, my friends call me Venkat", query="what is my name ?")

fprompt

'\n<s>[INST]<<SYS>>Answer the question only based on details in the context string below but dont mention about context. \nIf the answer is not in the context, reply "I dont know".\nContext: My give name is venkata ratnam, my friends call me Venkat\n<</SYS>>\nQuery: what is my name ? [/INST]\n'

In [12]:
inputs = tokenizer(fprompt, return_tensors="pt")

# Generate
generate_ids = model.generate(inputs.input_ids, max_length=150)
ans = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

print (ans)


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1477: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(



 [INST]<<SYS>>Answer the question only based on details in the context string below but dont mention about context. 
If the answer is not in the context, reply "I dont know".
Context: My give name is venkata ratnam, my friends call me Venkat
<</SYS>>
Query: what is my name ? [/INST]
Your name is Venkata Ratnam.


In [13]:
fprompt = prompt.format(context="My give name is venkata ratnam, my friends call me Venkat", query="what is the size of pacific ocean ?")

print (fprompt)
inputs = tokenizer(fprompt, return_tensors="pt")

generate_ids = model.generate(inputs.input_ids, max_length=150)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]



<s>[INST]<<SYS>>Answer the question only based on details in the context string below but dont mention about context. 
If the answer is not in the context, reply "I dont know".
Context: My give name is venkata ratnam, my friends call me Venkat
<</SYS>>
Query: what is the size of pacific ocean ? [/INST]



'\n [INST]<<SYS>>Answer the question only based on details in the context string below but dont mention about context. \nIf the answer is not in the context, reply "I dont know".\nContext: My give name is venkata ratnam, my friends call me Venkat\n<</SYS>>\nQuery: what is the size of pacific ocean ? [/INST]\nThe size of the Pacific Ocean is approximately 155.6 million square kilometers (60.1 million square miles).'

In [14]:
# Define the Text Splitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from chromadb.utils import embedding_functions
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

splits = text_splitter.split_documents(pages)
tokenizer.pad_token = tokenizer.eos_token
persist_directory = './docs/chroma/'
default_ef = embedding_functions.DefaultEmbeddingFunction()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
vectordb = Chroma.from_documents(
    documents=text_splitter.split_documents(pages),
    embedding=embeddings,
    persist_directory=persist_directory
)
print(vectordb._collection.count())

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

8


In [15]:
question = "what are different fields of computer vision ?"

# Similarity search with k = 5
docs = vectordb.similarity_search(question,k=5)

context = ""

for d in docs:
  context += d.page_content

fprompt = prompt.format(context=context, query=question)

inputs = tokenizer(fprompt, return_tensors="pt")

generate_ids = model.generate(inputs.input_ids, max_length=2000)
ans = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

print(ans[len(fprompt)-2:])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1477: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(



Based on the context provided, there are several fields of computer vision that are mentioned:

1. Object recognition and classification: This field involves developing algorithms and models that can recognize and classify objects within images or videos, such as animals, vehicles, or buildings.
2. Image segmentation: This field focuses on dividing an image into its constituent parts or objects, allowing for further analysis or processing of each segment.
3. Object detection: This field involves locating and identifying objects within an image or video, often using machine learning algorithms to detect and classify objects.
4. Facial recognition: This field involves developing algorithms that can identify and recognize individuals based on their facial features, often used in security or surveillance applications.
5. Tracking: This field involves tracking the movement of objects within a video or image sequence, often using machine learning algorithms to predict the future location of

In [16]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [23]:
question = "Am i subscribed to MTV channel ?"

context = """
Account name: John Doe
Subsciptions: Premium,
Premium subscription channels: MTV, CNN, Canal+, ESPN, Star sports
"""

fprompt = prompt.format(context=context, query=question)

inputs = tokenizer(fprompt, return_tensors="pt").to(device)

generate_ids = model.generate(inputs.input_ids, max_length=2000)
ans = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

print(ans[len(fprompt)-2:])

Yes, you are subscribed to the MTV channel.


In [24]:
question = "Am i subscribed to BBC ?"

context = """
Account name: John Doe
Subsciptions: Premium product
User is subscribed to channels MTV, CNN, Canal+, ESPN, Star sports only and other channels are not suscribed.
"""

fprompt = prompt.format(context=context, query=question)

inputs = tokenizer(fprompt, return_tensors="pt").to(device)

generate_ids = model.generate(inputs.input_ids, max_length=2000)
ans = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

print(ans[len(fprompt)-2:])

I don't know. According to the information provided in the context, the user is not subscribed to BBC.


In [22]:
question = "I took box c home, what are colors of balls i have ?"

context = """
There are three boxes A, B and C. Box A contains red and green color balls.
Box B contains red and yellow balls.
Box C contains green and blue balls.
"""

fprompt = prompt.format(context=context, query=question)

inputs = tokenizer(fprompt, return_tensors="pt").to(device)

generate_ids = model.generate(inputs.input_ids, max_length=2000)
ans = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

print(ans[len(fprompt)-2:])

The answer to your question is:

You have blue and green balls.
